In [16]:
# Clase Nodo original
class Nodo:
    def __init__(self, caracter, final):
        self.caracter = caracter
        self.indices = []
        self.es_final = final
        self.level = 0
        self.hijos = []

# Clase Trie original
class Trie:
    def __init__(self):
        self.raiz = Nodo("#", False)
    
    def insertar(self, nodo, cadena, i, og_i):
        length = len(cadena)
        if i < length:
            for h in nodo.hijos:
                if h.caracter == cadena[i]:
                    h.indices.append(og_i + i)
                    self.insertar(h, cadena, i + 1, og_i)
                    return
            
            hijo = Nodo(cadena[i], length - 1 == i)
            hijo.indices.append(og_i + i)
            hijo.level = nodo.level + 1
            nodo.hijos.append(hijo)
            self.insertar(hijo, cadena, i + 1, og_i)

# 1. Trie Diccionario para Autocompletado
class TrieDiccionario:
    def __init__(self):
        self.raiz = {'es_final': False, 'hijos': {}}
    
    def insertar_palabra(self, palabra):
        nodo = self.raiz
        for caracter in palabra:
            if caracter not in nodo['hijos']:
                nodo['hijos'][caracter] = {'es_final': False, 'hijos': {}}
            nodo = nodo['hijos'][caracter]
        nodo['es_final'] = True
    
    def buscar_completa(self, palabra):
        nodo = self.raiz
        for caracter in palabra:
            if caracter not in nodo['hijos']:
                return False
            nodo = nodo['hijos'][caracter]
        return nodo['es_final']
    
    def obtener_completaciones(self, prefijo):
        nodo = self.raiz
        for caracter in prefijo:
            if caracter not in nodo['hijos']:
                return []
            nodo = nodo['hijos'][caracter]
        
        completaciones = []
        self._recopilar_palabras(nodo, prefijo, completaciones)
        return completaciones
    
    def _recopilar_palabras(self, nodo, prefijo, resultado):
        if nodo['es_final']:
            resultado.append(prefijo)
        for caracter, hijo in nodo['hijos'].items():
            self._recopilar_palabras(hijo, prefijo + caracter, resultado)
    
    def autocompletar(self, cadena):
        if self.buscar_completa(cadena):
            print(f"La cadena {cadena} existe completa")
        else:
            completaciones = self.obtener_completaciones(cadena)
            if completaciones:
                print(f"Autocompletaciones para '{cadena}': {completaciones}")
            else:
                print(f"No se encontraron completaciones para '{cadena}'")

# 2. Suffix Trie para Búsqueda de Patrones  
class TrieExtendido(Trie):
    def __init__(self):
        super().__init__()
        self.cadena_original = ""
    
    def crear_desde_cadena(self, cadena):
        self.cadena_original = cadena
        sufijos = [cadena[i:] for i in range(len(cadena))]
        for i in range(len(sufijos)):
            self.insertar(self.raiz, sufijos[i], 0, i)
    
    def buscar_patron(self, patron):
        ocurrencias = []
        for i in range(len(self.cadena_original) - len(patron) + 1):
            if self.cadena_original[i:i+len(patron)] == patron:
                ocurrencias.append(i)
        
        if ocurrencias:
            print(f"Subcadena '{patron}': {[(inicio, inicio + len(patron) - 1) for inicio in ocurrencias]}")
        else:
            print(f"Patron '{patron}' no encontrado")

# Pruebas Trie Diccionario
palabras = ["casi", "casa", "cama", "camisa", "camara", "camion", "ave", "alce"]
trie_dict = TrieDiccionario()
for palabra in palabras:
    trie_dict.insertar_palabra(palabra)

print("=== TRIE DICCIONARIO ===")
casos_dict = ["cam", "casa", "xyz"]
for caso in casos_dict:
    trie_dict.autocompletar(caso)

# Pruebas Suffix Trie
cadena = "anabanana"
suffix_trie = TrieExtendido()
suffix_trie.crear_desde_cadena(cadena)

print("\n=== SUFFIX TRIE ===")
patrones = ["ana", "ban", "na"]
for patron in patrones:
    suffix_trie.buscar_patron(patron)

=== TRIE DICCIONARIO ===
Autocompletaciones para 'cam': ['cama', 'camara', 'camisa', 'camion']
La cadena casa existe completa
No se encontraron completaciones para 'xyz'

=== SUFFIX TRIE ===
Subcadena 'ana': [(0, 2), (4, 6), (6, 8)]
Subcadena 'ban': [(3, 5)]
Subcadena 'na': [(1, 2), (5, 6), (7, 8)]
